In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K

Using TensorFlow backend.


In [0]:
batch_size = 128
nb_classes = 10
nb_epoch = 100

img_rows, img_cols = 28, 28         # input image dimensions
pool_size = (2, 2)                  # size of pooling area for max pooling
prob_drop_conv = 0.2                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

In [0]:
weight_initializer = initializers.RandomNormal(stddev=0.01)

In [4]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('X_train original shape:', X_train.shape)

if K.image_dim_ordering() == 'th':
    # For Theano backend
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # For TensorFlow backend
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train original shape: (60000, 28, 28)


In [5]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
# Convolutional model
model = Sequential()

# conv1 layer
model.add(Convolution2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape, kernel_initializer=weight_initializer))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Dropout(prob_drop_conv))

# conv2 layer
model.add(Convolution2D(64, (3, 3), padding='same', activation='relu', kernel_initializer=weight_initializer))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Dropout(prob_drop_conv))

# conv3 layer
model.add(Convolution2D(128, (3, 3), padding='same', activation='relu', kernel_initializer=weight_initializer))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
model.add(Dense(625, activation='relu', kernel_initializer=weight_initializer))
model.add(Dropout(prob_drop_hidden))

# fc2 layer
model.add(Dense(10, activation='softmax', kernel_initializer=weight_initializer))

opt = RMSprop(lr=0.001, rho=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         73856     
__________

In [7]:
# Train
history = model.fit(X_train, Y_train, epochs=nb_epoch, batch_size=batch_size, shuffle=True, verbose=1)

Epoch 1/100
60000/60000 [==============================] - 10s 171us/step - loss: 0.3840 - acc: 0.8710
Epoch 2/100
60000/60000 [==============================] - 8s 137us/step - loss: 0.0979 - acc: 0.9693
Epoch 3/100
60000/60000 [==============================] - 8s 136us/step - loss: 0.0701 - acc: 0.9783
Epoch 4/100
60000/60000 [==============================] - 8s 137us/step - loss: 0.0560 - acc: 0.9829
Epoch 5/100
60000/60000 [==============================] - 8s 136us/step - loss: 0.0463 - acc: 0.9863
Epoch 6/100
60000/60000 [==============================] - 8s 136us/step - loss: 0.0412 - acc: 0.9872
Epoch 7/100
60000/60000 [==============================] - 8s 137us/step - loss: 0.0367 - acc: 0.9889
Epoch 8/100
60000/60000 [==============================] - 8s 137us/step - loss: 0.0358 - acc: 0.9893
Epoch 9/100
60000/60000 [==============================] - 8s 137us/step - loss: 0.0341 - acc: 0.9900
Epoch 10/100
60000/60000 [==============================] - 8s 137us/step - loss:

In [8]:
# Evaluate
evaluation = model.evaluate(X_test, Y_test, batch_size=256, verbose=1)
print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

10000/10000 [==============================] - 1s 52us/step
Summary: Loss over the test dataset: 0.03, Accuracy: 0.99
